In [1]:
fnames = io.l1a_filenames("dark", iterator=False, stage=False)

In [3]:
len(fnames)

6307

In [5]:
from IPython.parallel import Client
c = Client()

In [6]:
dview = c.direct_view()
lview = c.load_balanced_view()

In [7]:
l1a = io.L1AReader(fnames[0])

In [8]:
io.get_header_df(l1a.hdulist[0]).T.convert_objects()

,SIMPLE,BITPIX,NAXIS,NAXIS1,NAXIS2,NAXIS3,EXTEND,BLANK,FILENAME,CAPTURE,...,OBS_ID,INT_TIME,MCP_VOLT,MIR_DEG,N_FILL,BIN_TBL,SPA_OFS,SPA_SIZE,SPE_OFS,SPE_SIZE
0,True,32,3,256,10,21,True,-1,mvn_iuv_l1a_apoapse-orbit00626-fuvdark_2015012...,2015/025 Jan 25 15:33:02.76293UTC,...,8198,14.4,-1.83,32.901,0,"LINEAR 15,16 linear_0006",89,80,0,4


In [13]:
integration_df = l1a.Integration.T.set_index('ET')

In [17]:
integration_df['mean'] = l1a.img.mean(axis=(1,2))

In [18]:
integration_df['std'] = l1a.img.std(axis=(1,2))

In [33]:
for col in l1a.Observation.columns:
    print(col.name)

PRODUCT_ID
COLLECTION_ID
BUNDLE_ID
CODE_SVN_REVISION
ANC_SVN_REVISION
PRODUCT_CREATION_DATE
OBSERVATION_TYPE
MISSION_PHASE
TARGET_NAME
ORBIT_SEGMENT
ORBIT_NUMBER
SOLAR_LONGITUDE
GRATING_SELECT
KEYHOLE_SELECT
BIN_PATTERN_INDEX
CADENCE
INT_TIME
DUTY_CYCLE
CHANNEL
WAVELENGTH
WAVELENGTH_WIDTH
KERNELS


In [6]:
def process_fname(fname):
    import numpy as np
    l1a = io.L1AReader(fname)
    header_df = io.get_header_df(l1a.hdulist[0]).T.convert_objects()
    if header_df['NAXIS'] == 2:
        header_df['NAXIS3'] = np.nan
    header_df.drop(['XUV', 'OBS_ID', 'INT_TIME'], axis=1, inplace=True)
    df = pd.concat([header_df,
                    l1a.Engineering.T.convert_objects(),],
                   axis=1)
    df['primary_mean'] = l1a.img.mean()
    df['primary_std'] = l1a.img.std()
    if df.size != 61:
        return df, False
    else:
        savepath = io.save_to_hdf(df, fname, 'l1a_dark_scans')
        return df, True

In [7]:
results = lview.map_async(process_fname, fnames)

In [36]:
size_not_61 = []
dfs = []
for res in results:
    if not res[1]:
        size_not_61.append(res[0].FILENAME)
        dfs.append(res[0])

In [38]:
pd.concat(dfs, axis=0)

,AVERAGE,BIN_SHIFT,BIN_SHIFT_DIR,BIN_TBL,BIN_TYPE,BIN_X_ROW,BIN_Y_ROW,BITPIX,BLANK,BSCALE,...,START_TIME,START_TIME__SUB,STEP_INT,STEP_NUM,STEP_SIZE,SW_VER,TEST_PATTERN,TIME_FLAG,XUV,primary_mean
0,1922,0,0,"LINEAR 15,16 linear_0011",LINEAR,15,16,32,-1,NaN,...,480692579,27611,3,315,14,915,12,Synced,FUV,2663.203505
0,61733,4,1,"NON LINEAR 13,13 nonlin_0001",NON LINEAR,13,13,32,-1,NaN,...,471877268,49002,3,60,14,-1,12,Synced,MUV,42290.504348
0,20721,0,0,"NON LINEAR 12,12 nonlin_0001",NON LINEAR,12,12,32,-1,NaN,...,471616886,64371,3,315,14,-1,12,Synced,FUV,53130.544629
0,2413,0,0,"LINEAR 17,18 linear_0011",LINEAR,17,18,32,-1,NaN,...,478823096,65025,3,315,14,-1,12,Synced,MUV,3223.514683
0,1328,0,0,"LINEAR 5,6 linear_0011",LINEAR,5,6,32,-1,NaN,...,480734258,40827,255,0,-1,917,12,Synced,FUV,971.874286
0,7683,4,0,"LINEAR 7,8 linear_0011",LINEAR,7,8,32,-1,NaN,...,480326751,11588,3,101,13,911:913,12,Synced,MUV,53573.423197
0,9966,6,1,"LINEAR 17,18 linear_0011",LINEAR,17,18,32,-1,NaN,...,478940579,22379,3,315,14,911,12,Synced,MUV,61692.067361
0,9440,1,1,"LINEAR 15,16 linear_0011",LINEAR,15,16,32,-1,NaN,...,479731313,143,3,315,14,911,12,Synced,FUV,15389.216402
0,17650,0,0,"LINEAR 15,16 linear_0009",LINEAR,15,16,32,-1,NaN,...,478630201,7337,3,315,14,-1,12,Synced,FUV,32192.207490
0,32111,0,0,"LINEAR 0,19 linear_0011",LINEAR,0,19,32,-1,NaN,...,479222776,1124,255,0,-1,911,12,Synced,MUV,23181.817627


In [8]:
from iuvs import multitools

In [9]:
multitools.progress_display(results, fnames, sleep=20)

2.3 % done.
2.9 % done.
3.6 % done.
4.3 % done.
4.9 % done.
5.7 % done.
6.4 % done.
7.1 % done.
7.8 % done.
8.5 % done.
9.1 % done.
9.8 % done.
10.5 % done.
11.1 % done.
11.8 % done.
12.4 % done.
13.0 % done.
13.6 % done.
14.3 % done.
15.0 % done.
15.6 % done.
16.3 % done.
17.0 % done.
17.7 % done.
18.2 % done.
18.9 % done.
19.5 % done.
20.1 % done.
20.7 % done.
21.4 % done.
22.1 % done.
22.8 % done.
23.4 % done.
24.1 % done.
24.6 % done.
25.3 % done.
26.0 % done.
26.6 % done.
27.3 % done.
27.9 % done.
28.6 % done.
29.3 % done.
29.9 % done.
30.6 % done.
31.3 % done.
32.0 % done.
32.6 % done.
33.2 % done.
33.9 % done.
34.5 % done.
35.1 % done.
35.9 % done.
36.5 % done.
37.2 % done.
37.8 % done.
38.4 % done.
39.1 % done.
39.7 % done.
40.4 % done.
41.1 % done.
41.8 % done.
42.4 % done.
43.1 % done.
43.7 % done.
44.4 % done.
45.1 % done.
45.8 % done.
46.4 % done.
47.0 % done.
47.6 % done.
48.2 % done.
48.9 % done.
49.5 % done.
50.2 % done.
50.7 % done.
51.4 % done.
52.1 % done.
52.7 % done

In [10]:
import glob
h5fnames = glob.glob("/home/klay6683/output/l1a_dark_scans/*.h5")

In [11]:
len(h5fnames)

40421

In [12]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [13]:
dfs = []
for i,chunk in enumerate(chunker(h5fnames, 200)):
    print("Chunk {}".format(i))
    frames = []
    for fname in chunk:
        frames.append(pd.read_hdf(fname, 'df'))
    dfs.append(pd.concat(frames, ignore_index=True))

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Chunk 48
Chunk 49
Chunk 50
Chunk 51
Chunk 52
Chunk 53
Chunk 54
Chunk 55
Chunk 56
Chunk 57
Chunk 58
Chunk 59
Chunk 60
Chunk 61
Chunk 62
Chunk 63
Chunk 64
Chunk 65
Chunk 66
Chunk 67
Chunk 68
Chunk 69
Chunk 70
Chunk 71
Chunk 72
Chunk 73
Chunk 74
Chunk 75
Chunk 76
Chunk 77
Chunk 78
Chunk 79
Chunk 80
Chunk 81
Chunk 82
Chunk 83
Chunk 84
Chunk 85
Chunk 86
Chunk 87
Chunk 88
Chunk 89
Chunk 90
Chunk 91
Chunk 92
Chunk 93
Chunk 94
Chunk 95
Chunk 96
Chunk 97
Chunk 98
Chunk 99
Chunk 100
Chunk 101
Chunk 102
Chunk 103
Chunk 104
Chunk 105
Chunk 106
Chunk 107
Chunk 108
Chunk 109
Chunk 110


In [14]:
superdf = pd.concat(dfs, ignore_index=True)

In [15]:
superdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40421 entries, 0 to 40420
Data columns (total 63 columns):
AVERAGE               40421 non-null float64
BIN_SHIFT             40421 non-null int64
BIN_SHIFT_DIR         40421 non-null int64
BIN_TBL               40421 non-null object
BIN_TYPE              40421 non-null object
BIN_X_ROW             40421 non-null int64
BIN_Y_ROW             40421 non-null int64
BITPIX                40421 non-null int64
BLANK                 40421 non-null int64
BSCALE                494 non-null float64
BZERO                 494 non-null float64
CADENCE               40421 non-null float64
CAPTURE               40421 non-null object
CASE_TEMP             39927 non-null float64
CHECKSUM              40421 non-null float64
DATA_COMPRESSION      40421 non-null int64
DET_TEMP              39927 non-null float64
EXTEND                40421 non-null bool
FILENAME              40421 non-null object
FUV_BAD_PIXEL_MASK    40421 non-null int64
IMAGE_NUMBER      

In [18]:
from iuvs import calib

In [19]:
superdf.DET_TEMP = superdf.DET_TEMP.map(calib.convert_det_temp_to_C)
superdf.CASE_TEMP = superdf.CASE_TEMP.map(calib.convert_case_temp_to_C)

In [20]:
superdf.to_hdf('/home/klay6683/output/l1a_summary.h5','df')

In [21]:
meta.produce_summary_txt(superdf, 'l1a_summary.txt')